<div class="alert alert-info">

**Note:** Refer here for instructions to <a href="https://developers.arcgis.com/python/sample-notebooks/#Download-and-run-the-sample-notebooks">download and run this sample locally</a> on your computer

</div>

# Clone Portal users, groups and content

This sample notebook can be used for cloning a portal, from say, a staging to a production environment. It clones the users, groups and the content. It does not copy over services and data though, and works at the tier of portal items.

In [1]:
from arcgis.gis import GIS
from IPython.display import display

## Define the source and target portals
To start with, define the source and target portals. Connect to them using accounts with administrative privileges:

In [2]:
source = GIS("portal url", "username", "password")
target = GIS("target portal url", "username", "password")
target_admin_username = 'admin'

# Users
List the users in the source and target portals:

In [3]:
sourceusers = source.users.search()
sourceusers

[<User username:adams.powell>,
 <User username:admin>,
 <User username:allen.price>,
 <User username:anderson.bailey>,
 <User username:baker.long>,
 <User username:brown.rogers>,
 <User username:campbell.bryant>,
 <User username:carter.flores>,
 <User username:clark.ramirez>,
 <User username:davis.reed>,
 <User username:edwards.griffin>,
 <User username:esri_boundaries>,
 <User username:esri_demographics>,
 <User username:esri_livingatlas>,
 <User username:esri_nav>,
 <User username:evans.russell>,
 <User username:garcia.torres>,
 <User username:gonzalez.patterson>,
 <User username:green.perry>,
 <User username:hall.sanders>,
 <User username:harris.cox>,
 <User username:hernandez.wood>,
 <User username:hill.coleman>,
 <User username:jackson.cooper>,
 <User username:johnson.stewart>,
 <User username:jones.morris>,
 <User username:king.barnes>,
 <User username:lee.brooks>,
 <User username:lewis.watson>,
 <User username:lopez.henderson>,
 <User username:martin.howard>,
 <User username:mar

In [4]:
targetusers = target.users.search()
targetusers

[<User username:admin>,
 <User username:esri_boundaries>,
 <User username:esri_demographics>,
 <User username:esri_livingatlas>,
 <User username:esri_nav>,
 <User username:system_publisher>]

If source users are already in the target, run the following code to delete them:

In [5]:
# create a list of system accounts that should not be modified
systemusers = ['system_publisher', 'esri_nav', 'esri_livingatlas', 
               'esri_boundaries', 'esri_demographics']

### Remove existing users from target portal
Assign their content to admin account and delete the account

In [6]:
for srcuser in source_users:
    if not srcuser.username in system_users:
        #don't delete the account used to connect
        if srcuser.username != target_admin_username:
            try:
                targetusr = target.users.get(srcuser.username)
                if targetusr is not None:
                    print('Deleting user: ' + targetusr.fullName)
                    targetusr.reassign_to(target_admin_username)
                    targetusr.delete()
            except:
                print('User {} does not exist in Target Portal'.format(srcuser.username))

User admin does not exist in Target Portal


### Copy Users
Create a function that will accept connection to the target portal, user objects and password to create users with.

In [7]:
def copy_user(target, user, password):
    # See if the user has firstName and lastName properties
    try:
        first_name = user.firstName
        last_name = user.lastName
    except:
        # if not, split the fullName
        full_name = user.fullName
        first_name = full_name.split()[0]
        try:
            last_name = full_name.split()[1]
        except:
            last_name = 'NoLastName'

    try:
        # create user
        target_user = target.users.create(user.username, password, first_name, last_name,
                                          user.email, user.description, user.role)

        # update user properties
        target_user.update(user.access, user.preferredView,
                           user.description, user.tags, user.get_thumbnail_link(),
                           culture=user.culture, region=user.region)
        return target_user
    
    except:
        print("Unable to create user "+ user.username)
        return None

For each user in source portal, make a corresponding user in target portal:

In [8]:
ignore_list = [target_admin_username,'system_publisher', 'esri_nav', 'esri_livingatlas', 
               'esri_boundaries', 'esri_demographics']
for user in source_users:
    if not user.username in ignore_list:
        print("Creating user: " + user.username)
        copy_user(target, user, 'TestPassword@123')

Creating user: adams.powell
Creating user: allen.price
Creating user: anderson.bailey
Creating user: baker.long
Creating user: brown.rogers
Creating user: campbell.bryant
Creating user: carter.flores
Creating user: clark.ramirez
Creating user: davis.reed
Creating user: edwards.griffin
Creating user: evans.russell
Creating user: garcia.torres
Creating user: gonzalez.patterson
Creating user: green.perry
Creating user: hall.sanders
Creating user: harris.cox
Creating user: hernandez.wood
Creating user: hill.coleman
Creating user: jackson.cooper
Creating user: johnson.stewart
Creating user: jones.morris
Creating user: king.barnes
Creating user: lee.brooks
Creating user: lewis.watson
Creating user: lopez.henderson
Creating user: martin.howard
Creating user: martinez.peterson
Creating user: miller.cook
Creating user: mitchell.washington
Creating user: moore.bell
Creating user: nelson.hughes
Creating user: parker.alexander
Creating user: perez.butler
Creating user: phillips.gonzales
Creating u

Verify that users have been added to target portal:

In [9]:
target_users = target.users.search()
target_users

[<User username:adams.powell>,
 <User username:admin>,
 <User username:allen.price>,
 <User username:anderson.bailey>,
 <User username:baker.long>,
 <User username:brown.rogers>,
 <User username:campbell.bryant>,
 <User username:carter.flores>,
 <User username:clark.ramirez>,
 <User username:davis.reed>,
 <User username:edwards.griffin>,
 <User username:esri_boundaries>,
 <User username:esri_demographics>,
 <User username:esri_livingatlas>,
 <User username:esri_nav>,
 <User username:evans.russell>,
 <User username:garcia.torres>,
 <User username:gonzalez.patterson>,
 <User username:green.perry>,
 <User username:hall.sanders>,
 <User username:harris.cox>,
 <User username:hernandez.wood>,
 <User username:hill.coleman>,
 <User username:jackson.cooper>,
 <User username:johnson.stewart>,
 <User username:jones.morris>,
 <User username:king.barnes>,
 <User username:lee.brooks>,
 <User username:lewis.watson>,
 <User username:lopez.henderson>,
 <User username:martin.howard>,
 <User username:mar

Thus users have been successfully added to the target portal

# Groups

List the groups in the source and target portals:

In [10]:
source_groups = source.groups.search()
source_groups

[<Group title:"Basemaps" owner:admin>,
 <Group title:"Central Services" owner:admin>,
 <Group title:"Compliance" owner:admin>,
 <Group title:"Customer Service, Finance, Billing and Accounting" owner:admin>,
 <Group title:"Demographic Content" owner:admin>,
 <Group title:"Esri Boundary Layers" owner:esri_boundaries>,
 <Group title:"Esri Demographic Layers" owner:esri_demographics>,
 <Group title:"Living Atlas" owner:esri_livingatlas>,
 <Group title:"Living Atlas Analysis Layers" owner:esri_livingatlas>,
 <Group title:"Navigator Maps" owner:esri_nav>]

In [11]:
target_groups = target.groups.search()
target_groups

[<Group title:"Esri Boundary Layers" owner:esri_boundaries>,
 <Group title:"Esri Demographic Layers" owner:esri_demographics>,
 <Group title:"Featured Maps and Apps" owner:admin>,
 <Group title:"Living Atlas" owner:esri_livingatlas>,
 <Group title:"Living Atlas Analysis Layers" owner:esri_livingatlas>,
 <Group title:"Navigator Maps" owner:esri_nav>]

If source groups are already in the target, run the following code to delete them. If the group belongs to any of built-in user accounts, don't delete it.

In [12]:
groups_to_ignore = ['Basemaps']
for tg in target_groups:
    if tg.title not in groups_to_ignore:
        for sg in source_groups:
            if sg.title == tg.title and (not tg.owner in systemusers):
                print("Cleaning up group {} in target Portal...".format(tg.title))
                tg.delete()
                break

## Copy Groups

In [13]:
import tempfile

GROUP_COPY_PROPERTIES = ['title', 'description', 'tags', 'snippet', 'phone',
                         'access', 'isInvitationOnly']

def copy_group(target, source, group):
    """ Copy group to the target portal."""
    with tempfile.TemporaryDirectory() as temp_dir:
        # Create new groups with the subset of properties we want to
        # copy to the target portal. Handle switching between org and
        # public access when going from an org in a multitenant portal
        # and a single tenant portal
        target_group = {}
        
        for property_name in GROUP_COPY_PROPERTIES:
            target_group[property_name] = group[property_name]

        if target_group['access'] == 'org' and target.properties['portalMode'] == 'singletenant':
            target_group['access'] = 'public'
        elif target_group['access'] == 'public'\
             and source.properties['portalMode'] == 'singletenant'\
             and target.properties['portalMode'] == 'multitenant'\
             and 'id' in target.properties: # is org
            target_group['access'] = 'org'

        # Handle the thumbnail (if one exists)
        thumbnail_file = None
        if 'thumbnail' in group:
            target_group['thumbnail'] = group.download_thumbnail(temp_dir)

        # Create the group in the target portal
        copied_group = target.groups.create_from_dict(target_group)
        
         # Reassign all groups to correct owners, add users, and find shared items
        members = group.get_members()
        if not members['owner'] == target_admin_username:
            copied_group.reassign_to(members['owner'])
        if members['users']:
            copied_group.add_users(members['users'])
        return copied_group

For each group in source portal, make a corresponding group in target portal and a create a dictionary of source group id and corresponding group id in target portal:

In [14]:
copied_groups = {}
for group in sourcegroups:
    if group.title not in groups_to_ignore:
        if not group.owner in systemusers:
            print("Copying group: " + group.title)
            tgt_group = copy_group(target, source, group)
            copied_groups[group.groupid] = tgt_group.groupid

Copying group: Central Services
Copying group: Compliance
Copying group: Customer Service, Finance, Billing and Accounting
Copying group: Demographic Content


Verify that groups have been created in the target portal:

In [15]:
target_groups = target.groups.search()
target_groups

[<Group title:"Central Services" owner:admin>,
 <Group title:"Compliance" owner:admin>,
 <Group title:"Customer Service, Finance, Billing and Accounting" owner:admin>,
 <Group title:"Demographic Content" owner:admin>,
 <Group title:"Esri Boundary Layers" owner:esri_boundaries>,
 <Group title:"Esri Demographic Layers" owner:esri_demographics>,
 <Group title:"Featured Maps and Apps" owner:admin>,
 <Group title:"Living Atlas" owner:esri_livingatlas>,
 <Group title:"Living Atlas Analysis Layers" owner:esri_livingatlas>,
 <Group title:"Navigator Maps" owner:esri_nav>]

Print the mapping of source and target group ids:

In [16]:
copied_groups

{'0d4dc72e0dc94a24b36d04e01bb66df3': '3ad0380c85b94153a574089eaf4bffe8',
 '7d87ae57d57448f09433e5212b5ab982': '6ee7ab4bdd1d4a98a32f11e1446307d0',
 '856514710e37400fa6313f45c5c672a1': '9e5f3f26820642089e06fda63968731f',
 'ba5453f96ed44d598dfb3a16e1b21dee': '82ec68060e694f56a5462f983ec973be'}

With this part of the sample, we have successfully created users, groups and added the appropriate users to these groups.

# Items

Copying items consists of multiple steps. The following section of the sample does the following

 1. Create a dictionary of itemIds and `Item` objects for each user in each folder
 2. Prepare sharing information for each item

## Create a dictionary of itemIds and item objects
Do this for each user and each folder in the user account

In [17]:
source_items_by_id = {}
for user in source_users:
    if not user.username in system_users:  # ignore any 'system' Portal users
        print("Collecting item ids for {}...".format(user.username))
        user_content = user.items()
        # Copy item ids from root folder first
        for item in user_content:
            source_items_by_id[item.itemid] = item 
        # Copy item ids from folders next
        folders = user.folders
        for folder in folders:
            folder_items = user.items(folder=folder['title'])
            for item in folder_items:
                source_items_by_id[item.itemid] = item 

## Prepare sharing information for each item

In [18]:
for group in source_groups:
    if group.title not in groups_to_ignore:
        if not group.owner in system_users:
            target_group_id = copied_groups[group.groupid]
            for group_item in group.content():
                if not group_item.owner in system_users:
                    try:
                        item = source_items_by_id[group_item.itemid]
                        if item is not None:
                            if not 'groups'in item:
                                item['groups'] = []
                            item['groups'].append(target_group_id)
                    except:
                        print("Not found item : " + group_item.itemid)

### Print a mapping of item and its group membership

In [19]:
for key in source_items_by_id.keys():
    item = source_items_by_id[key]
    print("\n{:40s}".format(item.title), end = " # ")
    if 'groups' in item:
        print(item.access, end = " # ")
        print(item.groups, end = "")


Anderson Bailey response locations       # 
SampleWorldCities                        # 
DE                                       # 
CT                                       # 
NY                                       # 
Lewis Watson response locations          # 
WY                                       # 
PR                                       # 
IN                                       # 
Jackson Cooper response locations        # 
UT                                       # 
NM                                       # 
IN                                       # 
WY                                       # 
OR                                       # 
WA                                       # 
Hill Coleman response locations          # 
IL                                       # 
ME                                       # 
ME                                       # 
AL                                       # 
FL                                       # 
MA                             

## Copy Items

In [20]:
TEXT_BASED_ITEM_TYPES = frozenset(['Web Map', 'Feature Service', 'Map Service','Web Scene',
                                   'Image Service', 'Feature Collection', 
                                   'Feature Collection Template',
                                   'Web Mapping Application', 'Mobile Application', 
                                   'Symbol Set', 'Color Set',
                                   'Windows Viewer Configuration'])
ITEM_COPY_PROPERTIES = ['title', 'type', 'typeKeywords', 'description', 'tags',
                        'snippet', 'extent', 'spatialReference', 'name',
                        'accessInformation', 'licenseInfo', 'culture', 'url', ]

def copy_item(target, owner, folder, item):
    with tempfile.TemporaryDirectory() as temp_dir:
        copy_item = {}
        for property_name in ITEM_COPY_PROPERTIES:
            copy_item[property_name] = item[property_name]

        data_file = None
        if item.type in TEXT_BASED_ITEM_TYPES:
            # If its a text-based item, then read the text and add it to the request.
            if item.size > 0:
                text = item.get_data(False)
                #textstr = text.decode('utf-8')
                copy_item['text'] = text
        elif item.size > 0: # download data for all other types, not just item.type in FILE_BASED_ITEM_TYPES:
            # download data and add to the request as a file
            data_file = item.download(temp_dir)

        thumbnail_file = item.download_thumbnail(temp_dir)

        metadata_file = item.download_metadata(temp_dir)

        # Add the item to the target portal
        copied_item = target.content.add(copy_item, data_file, thumbnail_file, 
                                         metadata_file, owner, folder)

        return copied_item

In [21]:
RELATIONSHIP_TYPES = frozenset(['Map2Service', 'WMA2Code',
                                'Map2FeatureCollection', 'MobileApp2Code', 'Service2Data',
                                'Service2Service'])

def copy_relationships(target, copied_items, src_item, relationships, owner, folder):
    
    target_item_id = copied_items.get(src_item.itemid)
    if target_item_id is not None:
        target_item = target.content.get(target_item_id)

        for rel_type in RELATIONSHIP_TYPES:
            src_rel_items = src_item.related_items(rel_type)

            for src_rel_item in src_rel_items:
                print("***Found related items for " + src_rel_item.title)
                source_rel_id = src_rel_item.itemid

                # See if it's already been copied to the target
                target_rel_id = copied_items.get(source_rel_id)
                if not target_rel_id:
                    # If not, then copy it to the target - folder may have moved though?
                    target_rel_item = clone_item(target, owner, folder, src_rel_item)

                    if target_rel_item is not None:
                        # add relationship from target_item to copied item
                        result = target_item.add_relationship(target_rel_item, rel_type)

                        if not result:
                            print('Unable to add relationship from ' +  target_item.itemid + ' to ' + target_rel_item.itemid)
                    else:
                        print("@@@Error Cloning Item "+src_rel_item.title)

In [22]:
copied_items = {}
relationships = RELATIONSHIP_TYPES

for user in sourceusers:
    if not user.username in systemusers:
        print("**************\n"+user.username)
        usercontent = user.items()
        folders = user.folders
        for item in user_content:
            try:
                copied_item = copy_item(target, user, None, item)
                if copied_item is not None:
                    copied_items[item.itemid] = copied_item.itemid
                    # share the item
                    copied_item.share(item.access == 'public',
                                        item.access in ['org', 'public'],
                                        source_items_by_id[item.itemid].groups 
                                            if 'groups' in source_items_by_id[item.itemid]
                                            else None)
                    display(item)
                else:
                    print('Error copying ' + item.title)
            except:
                print("Error copying " + item.title)

        for folder in folders:
            target.content.create_folder(folder, user)
            folder_items = user.items(folder['title'])
            for item in folderitems:
                try:
                    copied_item = copy_item(target, user, folder['title'], item)
                    if copied_item is not None:
                        copied_items[item.itemid] = copied_item.itemid

                        # share the item
                        copied_item.share(item.access == 'public',
                                          item.access in ['org', 'public'],
                                          source_items_by_id[item.itemid].groups 
                                              if 'groups' in source_items_by_id[item.itemid]
                                              else None)
                    else:
                        print('Error copying ' + item.title)
                except:
                    print("Error copying " + item.title )

        # Copy the related items for this user (if specified)
        if relationships:
            for folder in folders:
                folder_items = user_content[folder]
                for item in folder_items:
                    try:
                        copy_relationships(target, copied_items, item, 
                                           relationships, user, folder)
                    except:
                        print("Error setting relationship for: " + item.title)


**************
adams.powell


<Item title:"Adams Powell response locations" type:Web Map owner:adams.powell>

<Item title:"SC" type:Feature Service owner:adams.powell>

<Item title:"SC" type:CSV owner:adams.powell>

**************
admin


...

<Item title:"WV" type:CSV owner:wright.ross>

**************
young.bennett


<Item title:"Young Bennett response locations" type:Web Map owner:young.bennett>

<Item title:"IA" type:CSV owner:young.bennett>

<Item title:"IA" type:Feature Service owner:young.bennett>